In [1]:
import sys, os
sys.path.append(os.path.realpath(".."))
os.environ["TF_XLA_FLAGS"]="--tf_xla_cpu_global_jit"

import tensorflow as tf
# tf.enable_eager_execution()
# tf.enable_v2_behavior()
# sess = tf.InteractiveSession()
# sess.as_default()

import util_funcs
from importlib import reload
import data_reader as read
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
import constants
import clinical_text_analysis as cta
import tsfresh
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, roc_curve
from os import path
# import predictSeizureConvExp as psce
import keras_models.dataGen as dg
from addict import Dict
# reload(psce)
from keras.utils import multi_gpu_model
import keras.optimizers
import ensembleReader as er
import time
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, log_loss
from functools import lru_cache
from importlib import reload
import constants

Using TensorFlow backend.


In [2]:
eds = er.EdfDatasetSegments(pre_cooldown=0, post_cooldown=0, sample_time=0, num_seconds=4, n_process=17)
train_label_files_segs = eds.get_train_split()
valid_label_files_segs = eds.get_valid_split()
test_label_files_segs = eds.get_test_split()

Starting 17 processes
retrieving: 0
Starting 17 processes
retrieving: 0
retrieving: 1000
retrieving: 2000
retrieving: 3000
starting custom_annotater
Starting 20 processes
retrieving: 0
retrieving: 1000
retrieving: 2000
starting custom_annotater
Starting 20 processes
retrieving: 0
starting custom_annotater
Starting 20 processes
retrieving: 0


In [3]:
n_process = 17
num_seconds=4
reload(er)
include_montage_channels=True
train_edss = er.EdfDatasetSegmentedSampler(segment_file_tuples=train_label_files_segs,  n_process=int(n_process), gap=num_seconds*pd.Timedelta(seconds=1), include_seizure_type=True, include_montage_channels=True, include_segment=True, shuffle=False)
valid_edss = er.EdfDatasetSegmentedSampler(segment_file_tuples=valid_label_files_segs,  n_process=int(n_process), gap=num_seconds*pd.Timedelta(seconds=1), include_seizure_type=True, include_montage_channels=include_montage_channels, include_segment=True, shuffle=False)
test_edss = er.EdfDatasetSegmentedSampler(segment_file_tuples=test_label_files_segs,  n_process=int(n_process), gap=num_seconds*pd.Timedelta(seconds=1), include_seizure_type=True, include_montage_channels=include_montage_channels, include_segment=True, shuffle=False)

In [4]:
debug_edss = er.EdfDatasetSegmentedSampler(segment_file_tuples=train_label_files_segs,  n_process=int(n_process), gap=num_seconds*pd.Timedelta(seconds=1), include_seizure_type=True, include_montage_channels=True, include_segment=True, shuffle=False, num_samples=1000)

In [5]:
train_edss.sampleInfo[0]

{'label': (False,
  'bckg',
  '/home/ms994/v1.5.0/edf/train/01_tcp_ar/131/00013182/s001_2015_10_07/00013182_s001_t000.edf'),
 'token_file_path': '/home/ms994/v1.5.0/edf/train/01_tcp_ar/131/00013182/s001_2015_10_07/00013182_s001_t000.edf',
 'sample_num': 0.0,
 'sample_width': Timedelta('0 days 00:00:04')}

In [6]:
from multiprocessing import Process, Manager

In [7]:
m = Manager()
resultsQ =  m.Queue(maxsize=30)
runQ = m.Queue()

In [8]:
n_process = 17

In [9]:
from tqdm import tqdm
def run(runQ, resultsQ, edss):
    for i in tqdm(iter(runQ.get, None)):
        resultsQ.put(edss[i])

In [10]:
p = [Process(target=run, args=(runQ, resultsQ, train_edss)) for i in range(n_process)]
[proc.start() for proc in p]
[runQ.put(i) for i in range(len(train_edss))]
[runQ.put(None) for i in range(n_process)]
print("setup q and processes")

2it [00:01,  1.16it/s]

setup q and processes


229it [02:54,  1.90it/s]Process Process-108:
Process Process-101:
Process Process-111:
Process Process-116:
Process Process-109:
Process Process-117:
Process Process-113:
Process Process-106:
Process Process-112:
Process Process-105:
Process Process-114:
Process Process-107:
Process Process-104:
Process Process-103:
Process Process-110:
Process Process-115:
Process Process-102:
Traceback (most recent call last):
  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent

  File "<ipython-input-9-42d3ee2fcfbb>", line 4, in run
    resultsQ.put(edss[i])
  File "/home/ms994/dbmi_eeg_clustering/ensembleReader.py", line 495, in get_montage_channel
    return read.gen_seizure_channel_labels(read.get_associated_lbl(indexData.token_file_path), indexData.sample_width).loc[pd.Timedelta(seconds=start)][constants.MONTAGE_COLUMNS]
  File "/home/ms994/dbmi_eeg_clustering/ensembleReader.py", line 512, in __getitem__
    max_length=self.gap)
  File "/home/ms994/dbmi_eeg_clustering/data_reader.py", line 647, in edf_eeg_2_df
    signal_data = reader.readSignal(i, start=start_count_native_freq, n=numStepsToRead)
  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/site-packages/pandas/core/reshape/concat.py", line 469, in _get_comb_axis
    sort=self.sort)
  File "/home/ms994/dbmi_eeg_clustering/data_reader.py", line 740, in get_per_channel_annotation
    curr_vals = pd.Series([curr_level, curr_sublevel, curr_start, curr_end, curr_channel, *curr_label_probs],

  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/site-packages/pandas/core/resample.py", line 998, in _downsample
    self._set_binner()
  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/site-packages/pandas/core/internals/construction.py", line 615, in sanitize_array
    subarr = maybe_convert_platform(data)
  File "/home/ms994/dbmi_eeg_clustering/data_reader.py", line 740, in get_per_channel_annotation
    curr_vals = pd.Series([curr_level, curr_sublevel, curr_start, curr_end, curr_channel, *curr_label_probs], index=["level", "sublevel", "start", "end", "channel", *symbols_list])
  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/site-packages/pandas/core/series.py", line 269, in __init__
    self._set_axis(0, index, fastpath=True)
  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/site-packages/pandas/core/resample.py", line 180, in _get_binner
    binner, bins, binlabels = self._get_binner_for_time()
  File "/home/ms994/min

  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/site-packages/pandas/core/dtypes/cast.py", line 843, in maybe_castable
    return arr.dtype.name not in _POSSIBLY_CAST_DTYPES
  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/site-packages/pandas/tseries/frequencies.py", line 239, in infer_freq
    return inferer.get_freq()
KeyboardInterrupt
  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/site-packages/pandas/tseries/frequencies.py", line 305, in get_freq
    if not self.is_monotonic or not self.index._is_unique:
  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/site-packages/pandas/core/arrays/datetimelike.py", line 912, in _is_unique
    return len(unique1d(self.asi8)) == len(self)
  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/site-packages/pandas/core/arrays/datetimes.py", line 327, in __init__
    if freq == "infer":
  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/site-packages/pandas/

  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/site-packages/pandas/core/indexes/datetimes.py", line 1170, in __getitem__
    result = self._data.__getitem__(key)
  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/site-packages/pandas/core/arrays/datetimelike.py", line 427, in __getitem__
    return self._box_func(val)
  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/site-packages/pandas/core/arrays/datetimes.py", line 513, in <lambda>
    return lambda x: Timestamp(x, freq=self.freq, tz=self.tz)
KeyboardInterrupt


In [31]:
resultsQ.qsize()

30

In [32]:
datum = resultsQ.get()

In [33]:
datum[1]

(False,
 'bckg',
 '/home/ms994/v1.5.0/edf/train/01_tcp_ar/131/00013182/s001_2015_10_07/00013182_s001_t000.edf',
 C3-CZ     0.0
 CZ-C4     0.0
 FP1-F7    0.0
 F8-T4     0.0
 F7-T3     0.0
 C4-T4     0.0
 FP2-F4    0.0
 T5-O1     0.0
 C4-P4     0.0
 F3-C3     0.0
 T3-T5     0.0
 A1-T3     0.0
 T4-A2     0.0
 FP2-F8    0.0
 FP1-F3    0.0
 T3-C3     0.0
 C3-P3     0.0
 T4-T6     0.0
 P4-O2     0.0
 P3-O1     0.0
 T6-O2     0.0
 F4-C4     0.0
 Name: 0 days 00:00:04, dtype: float64)

In [11]:
# Helperfunctions to make your feature definition more readable
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _int64_feature_list(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))
# Helperfunctions to make your feature definition more readable
def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _float_feature_list(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))


In [35]:
datum[1][3][constants.MONTAGE_COLUMNS].values.astype(np.int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [12]:
is_train = True
def parse_datum(datum):
    global is_train
    patient = read.parse_edf_token_path_structure(datum[1][2])[1]
    session =  read.parse_edf_token_path_structure(datum[1][2])[2]
    xData = datum[0]
    yData = datum[1]
    
    feature = { \
               'data': _float_feature_list(xData.reshape(-1)), \
               'label': _int64_feature_list([yData[0]]), \
               'subtypeLabel': _int64_feature_list([constants.SEIZURE_SUBTYPES.index(yData[1])]), \
               'patient': _int64_feature(read.getAllTrainPatients().index(patient) if is_train else 0), \
               'session': _int64_feature(read.getAllTrainSessions().index(session) if is_train else 0), \
               'montage': _int64_feature_list(yData[3][constants.MONTAGE_COLUMNS].values.astype(np.int))
              }
    return tf.train.Example(features=tf.train.Features(feature=feature))

In [13]:
def get_data(length):
    global resultsQ
    def gen():
        for i in range(length):
            yield parse_datum(resultsQ.get()).SerializeToString()
    return gen

In [14]:
dataset = tf.data.Dataset.from_generator(get_data(len(train_edss)), output_types=tf.string,  output_shapes=(tf.TensorShape([])))

In [15]:
writer = tf.data.experimental.TFRecordWriter("/n/scratch2/ms994/train_4s.tfr")

In [16]:
writer.write(dataset)

KeyboardInterrupt: 

In [ ]:
is_train = False

In [ ]:
m = Manager()
resultsQ =  m.Queue(maxsize=30)
runQ = m.Queue()
p = [Process(target=run, args=(runQ, resultsQ, valid_edss)) for i in range(n_process)]
[proc.start() for proc in p]
[runQ.put(i) for i in range(len(valid_edss))]
[runQ.put(None) for i in range(n_process)]
print("setup q and processes")

In [ ]:
dataset = tf.data.Dataset.from_generator(get_data(len(valid_edss)), output_types=tf.string,  output_shapes=(tf.TensorShape([])))
writer = tf.data.experimental.TFRecordWriter("/n/scratch2/ms994/valid_4s.tfr")
writer.write(dataset)

In [ ]:
m = Manager()
resultsQ =  m.Queue(maxsize=30)
runQ = m.Queue()
p = [Process(target=run, args=(runQ, resultsQ, test_edss)) for i in range(n_process)]
[proc.start() for proc in p]
[runQ.put(i) for i in range(len(test_edss))]
[runQ.put(None) for i in range(n_process)]
print("setup q and processes")
dataset = tf.data.Dataset.from_generator(get_data(len(test_edss)), output_types=tf.string,  output_shapes=(tf.TensorShape([])))
writer = tf.data.experimental.TFRecordWriter("/n/scratch2/ms994/test_4s.tfr")
writer.write(dataset)

In [ ]:
%%bash
scancel 2478576